In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` 
import org.apache.spark.sql._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}
import org.apache.spark.SparkContext  
import org.apache.spark.SparkConf
import org.apache.spark.sql.Column
import org.apache.spark.sql.types.{DataType, DateType, TimestampType}
import org.apache.spark.sql.functions._
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
def sc = spark.sparkContext
import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = org.joda.time.format.DateTimeFormatter@4e14d514
import scala.util.Try

import java.sql.{Connection, DriverManager, ResultSe

In [4]:
val frame = spark.read.option("header", true).option("inferSchema",true).csv("picsart_2.csv")
val df = frame.drop("advertising_id", "timezone", "duration", "app", "v").withColumn("date", frame("timestamp").cast(DateType))
df.printSchema()
def GroupCount(frame:DataFrame, groupCols:Array[String], countCols:Array[String]):DataFrame = {
    var groupCols1:Array[Column] = groupCols.map(x=>col(x))
    var countCols1:Array[Column] = countCols.map(c => countDistinct(col(c)).alias(c))
    var dfFiltered = frame.groupBy(groupCols.head, groupCols.tail : _*).agg(countCols1.head, countCols1.tail : _*)
    dfFiltered
    }
def find_avg(frame:DataFrame, avg_col:String):DataFrame = {
    var dfAvgCol = frame.agg(round(avg(col(avg_col)), 2))
    dfAvgCol
  }

root
 |-- device_id: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- user_id: long (nullable = true)
 |-- version: string (nullable = true)
 |-- date: date (nullable = true)



frame: DataFrame = [device_id: string, country_code: string ... 12 more fields]
df: DataFrame = [device_id: string, country_code: string ... 8 more fields]
defined function GroupCount
defined function find_avg

In [5]:
//5) find how many users did effect apply and what is descriptive statistics for it (outliers, avg, median)
val df_5 = df.filter(col("event_type")==="effect_apply").select("event_type", "device_id" )
val df_5_short = GroupCount(df_5, Array("event_type"), Array("device_id"))
//count, avg
df_5_short.show()
find_avg(df_5_short, "device_id").show()


+------------+---------+
|  event_type|device_id|
+------------+---------+
|effect_apply|       50|
+------------+---------+

+------------------------+
|round(avg(device_id), 2)|
+------------------------+
|                    50.0|
+------------------------+



df_5: DataFrame = [event_type: string, device_id: string]
df_5_short: DataFrame = [event_type: string, device_id: bigint]

In [ ]:
# outlier formula: outlier<lowerlevel[Q1-IQR*1.5] outlier>higher level[Q3+IQR*1.5] 

In [6]:
//6)* what first 5 events users do in their session start (should be used window functions, 
//find about it in the shared link above) - 5 events in array and % of users did it
//No general function yet.
var window = Window.partitionBy("session_id").orderBy("timestamp")
df.
withColumn("ev_1", lead("event_type", 1) over window).
withColumn("ev_2", lead("event_type", 2) over window).
withColumn("ev_3", lead("event_type", 3) over window).
withColumn("ev_4", lead("event_type", 4) over window).
withColumn("events",concat(col("event_type"),lit(","),col("ev_1"),lit(","),col("ev_2"),lit(","),col("ev_3"),lit(","),
        col("ev_4"))).select("session_id","events").groupBy("events").agg(countDistinct("session_id").as("count_5s")).
        orderBy(col("count_5s").desc).withColumn("percent_users",round(col("count_5s")/df.agg(countDistinct("session_id")).
        take(1)(0)(0).toString.toInt*100,2)).drop("count_5s").show()

+--------------------+-------------+
|              events|percent_users|
+--------------------+-------------+
|                null|        100.0|
|interstitial_ad_r...|         9.41|
|banner_ad_request...|         7.47|
|interstitial_ad_r...|         7.42|
|app_load,intersti...|         7.42|
|banner_ad_request...|         7.42|
|explore_open,app_...|         7.21|
|native_ad_waterfa...|         6.41|
|native_ad_respons...|         6.41|
|interstitial_ad_r...|         5.15|
|native_ad_respons...|          5.1|
|native_ad_request...|         5.02|
|app_open,explore_...|         5.02|
|explore_open,inte...|         4.85|
|shop_response,sho...|         4.81|
|effect_try,effect...|         4.72|
|banner_ad_request...|         4.39|
|native_ad_respons...|         4.26|
|shop_request,shop...|         4.22|
|shop_response,sho...|         4.01|
+--------------------+-------------+
only showing top 20 rows



window: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@586470c2

In [ ]:
//7)* What are the most important metrics when we want to describe one session of user in app (edited)
-session duration (avg session duration in a period of time)
-conversion rate of "event_try"/"event_apply"
-ranking of most used events
-